### This file is for processing the data from LUNA 16

In [1]:
import pandas as pd
import glob
import SimpleITK as sitk
import numpy as np
import collections
from PIL import Image, ImageDraw

df = pd.read_csv('annotations_with_malignancy.csv')
df.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm,mal_bool,mal_details,bboxLowX,bboxLowY,bboxLowZ,bboxHighX,bboxHighY,bboxHighZ,len_mal_details
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471,True,"[4, 2, 4, 2]",-131.896480,-178.259761,-299.800004,-125.451167,-172.458980,-296.200004,4
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735...,-69.175741,218.052625,-668.501273,4.441200,False,"[3, 2, 3]",-71.815939,215.709820,-670.900024,-66.722189,220.803570,-666.900024,3
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-67.459503,-247.744211,-211.095340,7.744222,True,"[5, 3, 4]",-73.027344,-253.847656,-213.100004,-61.074219,-242.558594,-209.500004,3
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-97.690876,-134.068054,-187.094883,6.690036,False,"[3, 3, 3]",-102.910156,-137.636719,-189.700005,-92.949219,-130.332031,-186.100005,3
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414...,69.529141,87.036153,-149.190113,10.824515,False,"[3, 3, 5, 2]",63.731247,80.468750,-154.000000,76.231247,93.750000,-144.000000,4


In [2]:
print(df.shape)

(1182, 14)


In [3]:
mal_bool = df['mal_bool']
mal_bool.value_counts()

False    804
True     378
Name: mal_bool, dtype: int64

In [4]:
candidate = pd.read_csv('candidates_V2.csv')
candidate.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


In [5]:
tmp = candidate['class']
tmp.value_counts()

0    753418
1      1557
Name: class, dtype: int64

In [6]:
tmp_data = []
for index, row in df.iterrows():
    seriesuid = row['seriesuid']
    coordX = row['coordX']
    coordY = row['coordY']
    coordZ = row['coordZ']
    mal_bool = row['mal_bool']
    
    
    # Append a new row to the new_data list
    tmp_data.append([seriesuid, coordX, coordY, coordZ, mal_bool])

# Create a new DataFrame with the specified columns
new_df = pd.DataFrame(tmp_data, columns=['seriesuid', 'coordX', 'coordY', 'coordZ', 'mal_bool'])

# Print the new DataFrame
new_df

,seriesuid,coordX,coordY,coordZ,mal_bool
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,True
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735...,-69.175741,218.052625,-668.501273,False
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-67.459503,-247.744211,-211.095340,True
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002...,-97.690876,-134.068054,-187.094883,False
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414...,69.529141,87.036153,-149.190113,False
...,...,...,...,...,...
1177,1.3.6.1.4.1.14519.5.2.1.6279.6001.299767339686...,-57.725518,73.779494,-147.415580,False
1178,1.3.6.1.4.1.14519.5.2.1.6279.6001.299767339686...,-72.287990,-77.481968,-160.486254,False
1179,1.3.6.1.4.1.14519.5.2.1.6279.6001.299767339686...,96.625954,-46.154823,-88.229432,False
1180,1.3.6.1.4.1.14519.5.2.1.6279.6001.297433269262...,33.104795,-63.116434,-108.725571,False


In [7]:
unique_value_count = new_df['coordZ'].nunique()
print(unique_value_count)

1182


In [8]:
# Create an empty dictionary to store the result
result_dict = {}

# Iterate through the DataFrame and populate the dictionary
for index, row in df.iterrows():
    seriesuid = row['seriesuid']
    coordinates = [row['coordX'], row['coordY'], row['coordZ']]
    # Check if the seriesuid is already in the dictionary, if not, add a new entry
    if seriesuid in result_dict:
        result_dict[seriesuid].append(coordinates)
    else:
        result_dict[seriesuid] = [coordinates]

# Print the resulting dictionary
print(result_dict)

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860': [[-128.6994211, -175.31927180000002, -298.38750639999995], [103.7836509, -211.9251487, -227.12125]], '1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735520890308424143898': [[-69.1757411, 218.0526254, -668.5012726], [81.09750567, 241.7018527, -701.3822918], [0.378580731, 213.6507752, -676.9320690000001]], '1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002726404418399209377': [[-67.45950309, -247.7442112, -211.0953396], [-97.69087642, -134.06805359999998, -187.0948829]], '1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414087561012670296643': [[69.52914115, 87.03615287, -149.1901128]], '1.3.6.1.4.1.14519.5.2.1.6279.6001.277452631455527999380186898011': [[76.95201463, -27.48605744, -148.58781000000002]], '1.3.6.1.4.1.14519.5.2.1.6279.6001.277662902666135640561346462196': [[106.9705181, 8.301864146, -139.54033590000003]], '1.3.6.1.4.1.14519.5.2.1.6279.6001.278010349511857248000260557753': [[-131.9599469, 55.69961281, -165.2127172]],

In [9]:
# Iterate through the dictionary and print key along with the count of coordinate sets
for key, value in result_dict.items():
    print(f"{key}: {len(value)}")

1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860: 2
1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735520890308424143898: 3
1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002726404418399209377: 2
1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414087561012670296643: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.277452631455527999380186898011: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.277662902666135640561346462196: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.278010349511857248000260557753: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.279300249795483097365868125932: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478: 7
1.3.6.1.4.1.14519.5.2.1.6279.6001.280125803152924778388346920341: 3
1.3.6.1.4.1.14519.5.2.1.6279.6001.281489753704424911132261151767: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.282512043257574309474415322775: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.283569726884265181140892667131: 4
1.3.6.1.4.1.14519.5.2.1.6279.6001.283733738239331719775105586296: 1
1.3.6.1.4.1.14519.5.2.1.6279.6001.28621753943435

In [10]:
len(result_dict)

600

In [11]:
for key in result_dict.keys():
    print(key, result_dict[key])

1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860 [[-128.6994211, -175.31927180000002, -298.38750639999995], [103.7836509, -211.9251487, -227.12125]]
1.3.6.1.4.1.14519.5.2.1.6279.6001.265775376735520890308424143898 [[-69.1757411, 218.0526254, -668.5012726], [81.09750567, 241.7018527, -701.3822918], [0.378580731, 213.6507752, -676.9320690000001]]
1.3.6.1.4.1.14519.5.2.1.6279.6001.276556509002726404418399209377 [[-67.45950309, -247.7442112, -211.0953396], [-97.69087642, -134.06805359999998, -187.0948829]]
1.3.6.1.4.1.14519.5.2.1.6279.6001.276710697414087561012670296643 [[69.52914115, 87.03615287, -149.1901128]]
1.3.6.1.4.1.14519.5.2.1.6279.6001.277452631455527999380186898011 [[76.95201463, -27.48605744, -148.58781000000002]]
1.3.6.1.4.1.14519.5.2.1.6279.6001.277662902666135640561346462196 [[106.9705181, 8.301864146, -139.54033590000003]]
1.3.6.1.4.1.14519.5.2.1.6279.6001.278010349511857248000260557753 [[-131.9599469, 55.69961281, -165.2127172]]
1.3.6.1.4.1.14519.5.2.1.6279

In [12]:
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))


In [13]:
# import os

# # 1. Specify the directory where you want to search for files
# directory = "E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2\\train"

# # 2. Define the prefix you want to search for
# prefix = 'marked_False_1-3-6-1-4-1-14519-5-2-1-6279-6001-100225287222365663678666836860_1_png'

# # 3. Use os.listdir() to get a list of files and directories in the specified directory
# files = os.listdir(directory)

# # 4. Loop through the files and check if the file name starts with the prefix
# for file in files:
#     if file.startswith(prefix):
#         # You've found a file with the desired prefix
#         print(f"Found file with prefix '{prefix}': {file}")

In [14]:
# import os
# root_dir = "E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2"

# # Define the filename you're looking for
# file_to_find = 'test.txt'
# file_path = ""

# # Iterate through the directory and its subdirectories
# for foldername, subfolders, filenames in os.walk(root_dir):
#     if file_to_find in filenames:
#         # The file is found
#         file_path = os.path.join(foldername, file_to_find)
#         print(f'Found {file_to_find} at {file_path}')
#         break  # If you want to stop searching after the first occurrence

# # If the loop completes without finding the file, it was not found
# else:
#     print(f'{file_to_find} not found in {root_dir} or its subdirectories')

In [15]:
# import shutil

# source_file = "C:\\Users\\oplab\\Desktop\\tmp.txt"  # The file you want to replace the existing file with
# destination_file = file_path  # The file you want to replace

# try:
#     shutil.copy2(source_file, destination_file)  # Copy the new file over the existing one
#     print(f"{destination_file} has been replaced with {source_file}.")
# except FileNotFoundError:
#     print(f"File {source_file} not found.")
# except PermissionError:
#     print(f"Permission denied. Make sure you have the necessary permissions to replace the file.")

In [16]:
final_list = []
for key in list(result_dict.keys()):
    # print(key, result_dict[key])
    # Glob finds all the pathnames matching a specified pattern.
    # In this case, it finds a file with the given format and assigns the path to `mhd_path`.
    mhd_path = glob.glob('../Luna_Data/subset*/{}.mhd'.format(key))
    # print(mhd_path)
    
    # Read the medical image from the specified path using SimpleITK.
    ct_mhd = sitk.ReadImage(mhd_path)
    # print(ct_mhd)
    if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
        ct_mhd = ct_mhd[...,0]
    # Convert the SimpleITK image to a NumPy array.
    hu_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    # print('hu_a',hu_a)
    
    # # Get the origin (starting point) of the image in XYZ coordinates.
    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())

    # # # Get the voxel size in each dimension (X, Y, Z) from the image spacing.
    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())

    # # # Get the direction matrix that defines the image orientation.
    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    # add for loop right here
    # converts XYZ coordinates to IRC (Index, Row, Column) coordinates.
    for i in range(len(result_dict[key])):
        center_xyz = result_dict[key][i]
        # print('i =', i)
        # print(center_xyz)
        
        # Find the true or false of the patient's nodule
        boolean = new_df.loc[new_df['coordX'] == center_xyz[0], 'mal_bool'].values[0]
        # print('boolean =', boolean)
        # print('===========')
        center_irc = xyz2irc(center_xyz, origin_xyz, vxSize_xyz, direction_a)
        
        # Print the calculated `center_irc`.
        # print(center_irc)
        # index, row, column
        index = center_irc[0]
        row = center_irc[1]
        col = center_irc[2]
        # print('original index =', index)
        
        # print(hu_a[index].shape)
        # print('index=', hu_a[index])
        final_list.append(hu_a[index])
        
        # Calculate the minimum and maximum values in the 78th slice of the CT image.
        # min_value = np.min(hu_a[index])
        # max_value = np.max(hu_a[index])
        # print('min =', min_value)
        # print('max =', max_value)

        # # Scale the pixel values of the 78th slice to the range [0, 255].
        # scaled_hu_a = (hu_a[index] - min_value) / (max_value - min_value) * 255

        # # Convert the scaled array to an integer data type (uint8).
        # scaled_hu_a = scaled_hu_a.astype(np.uint8)
        # print('scaled_hu_a =', scaled_hu_a)
        
        # # # 組合文件名，使用format方法將key和i插入文件名中
        # # filename = "./origin_data/marked_{}_{}_{}.png".format(boolean, key, i)
        # filename = "origin_{}_{}_{}.png".format(key, i, boolean)
        # slice_image = Image.fromarray(scaled_hu_a, mode='L')
        # # # 保存圖片，文件名使用組合的filename
        # slice_image.save(filename)

        # # =================================================
        
        # # add mark
        # # Create a drawing context on the image
        # draw = ImageDraw.Draw(slice_image)

        # # Define the coordinates to mark (row 212, column 45) as a red rectangle
        # x1, y1, x2, y2 = col-10, row-10, col+10, row+10  # Adjust these coordinates as needed
        # # print(x1, x2, y1, y2)
        # # Define the outline color
        # outline_color = (0, 0, 255)  

        # # Draw a white rectangle on the image to mark the specific row and column
        # draw.rectangle([x1, y1, x2, y2], outline="white", width=1)  # Increase width for better visibility

        # # Save the marked slice as a PNG
        # mark_filename = "marked_{}_{}_{}.png".format(key, i, boolean)
        # # 保存圖片，文件名使用組合的filename
        # slice_image.save(mark_filename)
        # # print('=============')

stack_list = np.stack(final_list)
print(stack_list.shape)

(1182, 512, 512)


In [19]:
print(stack_list)

[[[-1022. -1008.  -971. ...  -977.  -993. -1013.]
  [-1022. -1022. -1010. ... -1005. -1016. -1009.]
  [-1016. -1018. -1011. ... -1023. -1013. -1008.]
  ...
  [ -904.  -894.  -901. ...   337.   243.    65.]
  [ -894.  -891.  -905. ...    36.  -177.  -435.]
  [ -887.  -901.  -906. ...  -463.  -665.  -850.]]

 [[-1002.  -985.  -985. ...  -984.  -991.  -999.]
  [-1024.  -996.  -982. ...  -997.  -997.  -998.]
  [-1024. -1017.  -997. ...  -986.  -979. -1002.]
  ...
  [ -869.  -889.  -896. ...   420.   368.   233.]
  [ -896.  -904.  -903. ...   216.    41.  -203.]
  [ -901.  -902.  -905. ...  -220.  -455.  -689.]]

 [[-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]
  [-1000. -1000. -1000. ... -1000. -1000. -1000.]
  ...
  [ -982.  -975.  -964. ...  -835.  -921.  -989.]
  [ -963.  -945.  -939. ...  -432.  -601.  -808.]
  [ -947.  -954.  -959. ...    13.  -165.  -455.]]

 ...

 [[-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -10

In [ ]:
# import os

# origindir = "E:\\LUNA\\yolo\\origin_data"
# targetdirs = ["E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2\\train\\images", "E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2\\test\\images", "E:\\LUNA\\nodule_detection\\yolo\\nodule-detection-2\\valid\\images"]

# ori_files = os.listdir(origindir)

# target_files = []
# for targetdir in targetdirs:
#     target_files.append(os.listdir(targetdir))

# for file in ori_files:
#     last_dot_index = file.rfind(".")
#     prefix = ""
#     if last_dot_index != -1:
#         # Split the string into two parts: everything before the last dot and the file extension
#         before_last_dot = file[:last_dot_index]
#         file_extension = file[last_dot_index + 1:]

#         # Replace the last dot with an underscore
#         prefix = before_last_dot + "_" + file_extension
#         prefix = prefix.replace(".", "-")
#         print(prefix)
#     for file in target_files[1]:
#         if file.startswith(prefix):
#             # You've found a file with the desired prefix
#             print(f"Found file with prefix '{prefix}': {file}")

In [ ]:
# # Find the last dot (.) in the string
# last_dot_index = input_string.rfind(".")

# if last_dot_index != -1:
#     # Split the string into two parts: everything before the last dot and the file extension
#     before_last_dot = input_string[:last_dot_index]
#     file_extension = input_string[last_dot_index + 1:]

#     # Replace the last dot with an underscore
#     modified_string = before_last_dot + "_" + file_extension

#     print(modified_string)
# else:
#     # If there is no dot in the string, do something else or handle the case as needed
#     print("No dot found in the input string")